In [6]:
#Installations Required to Run This Code
# 1. pyaudio
# For capturing live audio from the microphone:
# pip install pyaudio

# 2. wave
# wave is a standard library module, so you don't need to install it separately. It comes with Python's standard library.

# 3. pydub
# For audio manipulation and playback:
# pip install pydub

# 4. IPython
# For audio playback in Jupyter notebooks:
# pip install ipython

# 5. matplotlib
# For plotting the audio waveform (if using Jupyter or similar):
#pip install matplotlib

# 6. sounddevice
# For playing back audio:
# pip install sounddevice

# 7. soundfile
# For saving and reading audio files:
# pip install soundfile

# 8. pedalboard
# For applying audio effects:
# pip install pedalboard

# 9. noisereduce
# For noise reduction:
# pip install noisereduce


# Summary of Installation Commands
# To install all the required packages, you can use the following command:
# pip install pyaudio pydub ipython matplotlib sounddevice soundfile pedalboard noisereduce

In [7]:
#After Running this to stop Recording Audio You Have To Create Keyboard Interrupt
#In Jupyter Key Board Interrupt is "PRESS "i" TWO Times"
#In Pycharm it is "Ctrl+C"

import pyaudio
import wave
from pydub import AudioSegment
from pydub.playback import play

audio=pyaudio.PyAudio()
stream=audio.open(format=pyaudio.paInt16,channels=1,rate=44100,input=True,frames_per_buffer=1024)
frames=[]
print("Recording")
try:
    while True:
        data=stream.read(1024)
        frames.append(data)
except KeyboardInterrupt:
    pass
stream.stop_stream()
stream.close()
audio.terminate()

sound_file=wave.open("myrec1.wav","wb")
sound_file.setnchannels(1)
sound_file.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
sound_file.setframerate(44100)
sound_file.writeframes(b''.join(frames))
sound_file.close()

file_path = "myrec1.wav"
audio = AudioSegment.from_file(file_path)
print("playing")
play(audio)
print("Successful")

Recording
playing
Successful


In [8]:
#Play Your Audio For Reference
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
filename="myrec1.wav"
plt.figure(figsize=(14,5))
ipd.Audio(filename)

<Figure size 1400x500 with 0 Axes>

In [9]:
#This Code used to Reduce Noise
#Adjust threshold Values according to your requirement

import sounddevice as sd
import soundfile as sf
from pedalboard.io import AudioFile
from pedalboard import *
import noisereduce as nr

sr = 44100
with AudioFile('myrec1.wav').resampled_to(sr) as f:
    audio = f.read(f.frames)

# Reduce noise
reduced_noise = nr.reduce_noise(y=audio, sr=sr, stationary=True, prop_decrease=0.75)

# Define the audio processing chain
board = Pedalboard([
    NoiseGate(threshold_db=15, ratio=2, release_ms=200),
    Compressor(threshold_db=-16, ratio=2.5),
    LowShelfFilter(cutoff_frequency_hz=400, gain_db=5, q=1),
    Gain(gain_db=10)
])

# Apply effects
effected = board(reduced_noise, sr)

# Play the processed audio
print('Playing')
sd.play(effected[0], sr)
sd.wait() 
print('Finished.')

# Save the processed audio to a new file
sf.write('noiseless.wav', effected[0], sr)
print('Processed audio saved as noiseless.wav')


Playing
Finished.
Processed audio saved as noiseless.wav


In [10]:
#Play Noise Less Audio
ipd.Audio("noiseless.wav")